# Data Cleaning
### Data from Estimation of the warfarin dose with clinical and pharmacogenetic data (PMID:19228618): https://api.pharmgkb.org/v1/download/submission/553247439
---

## Load Data

In [1]:
from tqdm import tqdm
import pandas as pd
import warnings

warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

In [2]:
org_df = pd.read_excel('../PS206767-553247439.xls', sheet_name='Subject Data')
org_df

,PharmGKB Subject ID,PharmGKB Sample ID,Project Site,Gender,Race (Reported),Race (OMB),Ethnicity (Reported),Ethnicity (OMB),Age,Height (cm),...,VKORC1 QC genotype: -4451 C>A (861); Chr16:31018002; rs17880887; A/C,CYP2C9 consensus,VKORC1 -1639 consensus,VKORC1 497 consensus,VKORC1 1173 consensus,VKORC1 1542 consensus,VKORC1 3730 consensus,VKORC1 2255 consensus,VKORC1 -4451 consensus,Comments regarding Project Site Dataset
0,PA135312261,PA135312629,1,male,White,White,not Hispanic or Latino,not Hispanic or Latino,60 - 69,193.040,...,NaN,*1/*1,A/G,G/T,NaN,C/G,A/G,NaN,NaN,Project 1:
1,PA135312262,PA135312630,1,female,White,White,not Hispanic or Latino,not Hispanic or Latino,50 - 59,176.530,...,C/C,*1/*1,A/A,G/T,T/T,C/C,G/G,T/T,C/C,Warfarin Therapeutic Dose Definition:
2,PA135312263,PA135312631,1,female,White,White,not Hispanic or Latino,not Hispanic or Latino,40 - 49,162.560,...,NaN,*1/*1,G/G,T/T,NaN,G/G,A/G,NaN,NaN,The dose (unchanged for 6 days) that yielded a...
3,PA135312264,PA135312632,1,male,White,White,not Hispanic or Latino,not Hispanic or Latino,60 - 69,182.245,...,NaN,*1/*1,A/G,G/T,NaN,C/G,G/G,NaN,NaN,NaN
4,PA135312265,PA135312633,1,male,White,White,not Hispanic or Latino,not Hispanic or Latino,50 - 59,167.640,...,NaN,*1/*3,A/G,T/T,NaN,C/G,A/G,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5695,PA152407681,PA152407969,21,male,White,White,not Hispanic or Latino,not Hispanic or Latino,20 - 29,185.420,...,NaN,*1/*1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5696,PA152407682,PA152407970,21,female,White,White,not Hispanic or Latino,not Hispanic or Latino,70 - 79,160.020,...,NaN,*1/*3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5697,PA152407683,PA152407971,21,male,White,White,not Hispanic or Latino,not Hispanic or Latino,60 - 69,187.960,...,NaN,*1/*1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5698,PA152407684,PA152407972,21,male,White,White,not Hispanic or Latino,not Hispanic or Latino,60 - 69,177.800,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


---
## Inspection

In [3]:
org_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5700 entries, 0 to 5699
Data columns (total 68 columns):
 #   Column                                                                Non-Null Count  Dtype  
---  ------                                                                --------------  -----  
 0   PharmGKB Subject ID                                                   5700 non-null   object 
 1   PharmGKB Sample ID                                                    5700 non-null   object 
 2   Project Site                                                          5700 non-null   int64  
 3   Gender                                                                5696 non-null   object 
 4   Race (Reported)                                                       5194 non-null   object 
 5   Race (OMB)                                                            5700 non-null   object 
 6   Ethnicity (Reported)                                                  4461 non-null   object 
 7

In [4]:
cols_keeps = [
    "PharmGKB Subject ID", # Index
    "Gender", # Male, Female or not known = -99
    "Age",
    "Height (cm)",
    "Weight (kg)",

    # Warfarin
    "Indication for Warfarin Treatment", # DVT = 1, PE = 2, Afib/flutter = 3, Heart Valve = 4, Cardiomyopathy/LV Dilation = 5, Stroke = 6, Post-Orthopedic = 7, Other = 8 or NA; multiple indications are separated by semi-colons
    "Subject Reached Stable Dose of Warfarin", # yes = 1, no = 0 or not known = NA
    "Therapeutic Dose of Warfarin", # Dose given in milligrams/week
    "INR on Reported Therapeutic Dose of Warfarin", # International Normalized Ratio on the Therapeutic Dose of Warfarin Reported Above
    
    # Diseases & Medications & Behaviors
    "Diabetes", # yes = 1, not present = 0 or not known = NA
    "Congestive Heart Failure and/or Cardiomyopathy", # yes = 1, not present = 0 or not known = NA
    "Valve Replacement", # yes = 1, not present = 0 or not known = NA
    "Aspirin", # yes = 1, not present = 0 or not known = NA
    "Acetaminophen or Paracetamol (Tylenol)", # yes = 1, not present = 0 or not known = NA
    "Simvastatin (Zocor)", # yes = 1, not present = 0 or not known = NA
    "Atorvastatin (Lipitor)", # yes = 1, not present = 0 or not known = NA
    "Fluvastatin (Lescol)", # yes = 1, not present = 0 or not known = NA
    "Lovastatin (Mevacor)", # yes = 1, not present = 0 or not known = NA
    "Pravastatin (Pravachol)", # yes = 1, not present = 0 or not known = NA
    "Rosuvastatin (Crestor)", # yes = 1, not present = 0 or not known = NA
    "Cerivastatin (Baycol)", # yes = 1, not present = 0 or not known = NA
    "Amiodarone (Cordarone)", # yes = 1, not present = 0 or not known = NA
    "Carbamazepine (Tegretol)", # yes = 1, not present = 0 or not known = NA
    "Phenytoin (Dilantin)", # yes = 1, not present = 0 or not known = NA
    "Rifampin or Rifampicin", # yes = 1, not present = 0 or not known = NA
    "Sulfonamide Antibiotics", # Includes Septra, Bactrim, Cotrim and Sulfatrim; yes = 1, not present = 0 or not known = NA
    "Macrolide Antibiotics", # Includes erythromycin, azithromycin, and clarithromycin; yes = 1, not present = 0 or not known = NA
    "Anti-fungal Azoles", # Includes ketoconazole, fluconazole, itraconazole, metronidazole, etc. Please do not include other drugs that end in "azole" such as omeprazole or metronidazole; yes = 1, not present = 0 or not known = NA
    "Current Smoker", # yes = 1, not present = 0 or not known = NA
]

In [5]:
dirty_df = org_df[cols_keeps]
dirty_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5700 entries, 0 to 5699
Data columns (total 29 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   PharmGKB Subject ID                             5700 non-null   object 
 1   Gender                                          5696 non-null   object 
 2   Age                                             5658 non-null   object 
 3   Height (cm)                                     4554 non-null   float64
 4   Weight (kg)                                     5413 non-null   float64
 5   Indication for Warfarin Treatment               5002 non-null   object 
 6   Subject Reached Stable Dose of Warfarin         5651 non-null   float64
 7   Therapeutic Dose of Warfarin                    5528 non-null   float64
 8   INR on Reported Therapeutic Dose of Warfarin    4968 non-null   float64
 9   Diabetes                                 

---
## Missing Data

In [6]:
# drop all None values for core columns(0~8)
dirty_df.dropna(subset=dirty_df.columns[0:9], inplace=True)
dirty_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3989 entries, 0 to 5699
Data columns (total 29 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   PharmGKB Subject ID                             3989 non-null   object 
 1   Gender                                          3989 non-null   object 
 2   Age                                             3989 non-null   object 
 3   Height (cm)                                     3989 non-null   float64
 4   Weight (kg)                                     3989 non-null   float64
 5   Indication for Warfarin Treatment               3989 non-null   object 
 6   Subject Reached Stable Dose of Warfarin         3989 non-null   float64
 7   Therapeutic Dose of Warfarin                    3989 non-null   float64
 8   INR on Reported Therapeutic Dose of Warfarin    3989 non-null   float64
 9   Diabetes                                      

In [7]:
# For all diseases, if value is unknown, we assume it is 0(FALSE)
dirty_df.fillna(0, inplace=True)
dirty_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3989 entries, 0 to 5699
Data columns (total 29 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   PharmGKB Subject ID                             3989 non-null   object 
 1   Gender                                          3989 non-null   object 
 2   Age                                             3989 non-null   object 
 3   Height (cm)                                     3989 non-null   float64
 4   Weight (kg)                                     3989 non-null   float64
 5   Indication for Warfarin Treatment               3989 non-null   object 
 6   Subject Reached Stable Dose of Warfarin         3989 non-null   float64
 7   Therapeutic Dose of Warfarin                    3989 non-null   float64
 8   INR on Reported Therapeutic Dose of Warfarin    3989 non-null   float64
 9   Diabetes                                      

In [8]:
dirty_df.reset_index(drop=True, inplace=True)
dirty_df

,PharmGKB Subject ID,Gender,Age,Height (cm),Weight (kg),Indication for Warfarin Treatment,Subject Reached Stable Dose of Warfarin,Therapeutic Dose of Warfarin,INR on Reported Therapeutic Dose of Warfarin,Diabetes,...,Rosuvastatin (Crestor),Cerivastatin (Baycol),Amiodarone (Cordarone),Carbamazepine (Tegretol),Phenytoin (Dilantin),Rifampin or Rifampicin,Sulfonamide Antibiotics,Macrolide Antibiotics,Anti-fungal Azoles,Current Smoker
0,PA135312261,male,60 - 69,193.040,115.70,7,1.0,49.00,2.60,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,PA135312262,female,50 - 59,176.530,144.20,7,1.0,42.00,2.15,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,PA135312263,female,40 - 49,162.560,77.10,7,1.0,53.00,1.90,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,PA135312264,male,60 - 69,182.245,90.70,7,1.0,28.00,2.40,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,PA135312265,male,50 - 59,167.640,72.60,7,1.0,42.00,1.90,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3984,PA152407680,female,60 - 69,165.100,86.36,8,1.0,35.00,3.00,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3985,PA152407682,female,70 - 79,160.020,55.91,3,1.0,27.51,2.80,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3986,PA152407683,male,60 - 69,187.960,97.73,3; 8,1.0,57.47,2.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3987,PA152407684,male,60 - 69,177.800,87.27,8,1.0,70.00,2.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


---
## One-hot Encoder

In [9]:
# None numeric columns:
# Indication for Warfarin Treatment - DVT = 1, PE = 2, Afib/flutter = 3, Heart Valve = 4, Cardiomyopathy/LV Dilation = 5, Stroke = 6, Post-Orthopedic = 7, Other = 8 or NA; multiple indications are separated by semi-colons
# Gender - Male, Female  
# Age - n~n+9

from sklearn.preprocessing import OneHotEncoder

#### Gender & Age

In [10]:
encoder = OneHotEncoder(sparse_output=False)
encoded = encoder.fit_transform(dirty_df[['Gender', 'Age']])

In [11]:
encoded_df = pd.DataFrame(
    encoded,
    columns=encoder.get_feature_names_out(['Gender', 'Age']),
    index=dirty_df.index
)
encoded_df

,Gender_female,Gender_male,Age_10 - 19,Age_20 - 29,Age_30 - 39,Age_40 - 49,Age_50 - 59,Age_60 - 69,Age_70 - 79,Age_80 - 89,Age_90+
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
3984,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3985,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3986,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3987,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


#### Indication for Warfarin Treatment

In [12]:
mapping = {
    '1': 'DVT',
    '2': 'PE',
    '3': 'Afib/flutter',
    '4': 'Heart Valve',
    '5': 'Cardiomyopathy/LV Dilation',
    '6': 'Stroke',
    '7': 'Post-Orthopedic',
    '8': 'Other'
}

In [13]:
indications = dirty_df['Indication for Warfarin Treatment'].to_list()

In [14]:
for i, elm in tqdm(enumerate(indications)):
    elms = str(elm).replace(' ', '').split(';')
    
    for num, col_name in mapping.items():
        if num in elms:
            dirty_df.loc[i, col_name] = 1
        else:
            dirty_df.loc[i, col_name] = 0

3989it [00:01, 3607.43it/s]


#### Combine One-hot Result

In [15]:
dirty_df = pd.concat([dirty_df, encoded_df], axis=1)
dirty_df.drop(columns=['Age', 'Gender', 'Indication for Warfarin Treatment'], inplace=True)
dirty_df

,PharmGKB Subject ID,Height (cm),Weight (kg),Subject Reached Stable Dose of Warfarin,Therapeutic Dose of Warfarin,INR on Reported Therapeutic Dose of Warfarin,Diabetes,Congestive Heart Failure and/or Cardiomyopathy,Valve Replacement,Aspirin,...,Gender_male,Age_10 - 19,Age_20 - 29,Age_30 - 39,Age_40 - 49,Age_50 - 59,Age_60 - 69,Age_70 - 79,Age_80 - 89,Age_90+
0,PA135312261,193.040,115.70,1.0,49.00,2.60,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,PA135312262,176.530,144.20,1.0,42.00,2.15,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,PA135312263,162.560,77.10,1.0,53.00,1.90,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,PA135312264,182.245,90.70,1.0,28.00,2.40,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,PA135312265,167.640,72.60,1.0,42.00,1.90,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3984,PA152407680,165.100,86.36,1.0,35.00,3.00,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3985,PA152407682,160.020,55.91,1.0,27.51,2.80,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3986,PA152407683,187.960,97.73,1.0,57.47,2.00,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3987,PA152407684,177.800,87.27,1.0,70.00,2.00,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


---
## To Float & Rounding to 2 Decimal

In [16]:
for col in dirty_df.select_dtypes(include='number').columns:
    dirty_df[col] = dirty_df[col].astype(float).round(2)

---
## Saving to csv

In [17]:
clean_df = dirty_df.copy(deep=True)
clean_df

,PharmGKB Subject ID,Height (cm),Weight (kg),Subject Reached Stable Dose of Warfarin,Therapeutic Dose of Warfarin,INR on Reported Therapeutic Dose of Warfarin,Diabetes,Congestive Heart Failure and/or Cardiomyopathy,Valve Replacement,Aspirin,...,Gender_male,Age_10 - 19,Age_20 - 29,Age_30 - 39,Age_40 - 49,Age_50 - 59,Age_60 - 69,Age_70 - 79,Age_80 - 89,Age_90+
0,PA135312261,193.04,115.70,1.0,49.00,2.60,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,PA135312262,176.53,144.20,1.0,42.00,2.15,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,PA135312263,162.56,77.10,1.0,53.00,1.90,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,PA135312264,182.24,90.70,1.0,28.00,2.40,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,PA135312265,167.64,72.60,1.0,42.00,1.90,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3984,PA152407680,165.10,86.36,1.0,35.00,3.00,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3985,PA152407682,160.02,55.91,1.0,27.51,2.80,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3986,PA152407683,187.96,97.73,1.0,57.47,2.00,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3987,PA152407684,177.80,87.27,1.0,70.00,2.00,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [18]:
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3989 entries, 0 to 3988
Data columns (total 45 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   PharmGKB Subject ID                             3989 non-null   object 
 1   Height (cm)                                     3989 non-null   float64
 2   Weight (kg)                                     3989 non-null   float64
 3   Subject Reached Stable Dose of Warfarin         3989 non-null   float64
 4   Therapeutic Dose of Warfarin                    3989 non-null   float64
 5   INR on Reported Therapeutic Dose of Warfarin    3989 non-null   float64
 6   Diabetes                                        3989 non-null   float64
 7   Congestive Heart Failure and/or Cardiomyopathy  3989 non-null   float64
 8   Valve Replacement                               3989 non-null   float64
 9   Aspirin                                  

In [19]:
clean_df.to_csv("../cleaned_data.csv", index=False)